In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()


In [ ]:
def polite_function():
    """Return a polite greeting."""
    return "Hello! How are you doing?"


def impolite_function():
    """Return an impolite greeting."""
    return "Hey! How are you doing?"


import inspect
import typing


def describe_function(func):
    # Extract the signature of the function
    signature = inspect.signature(func)
    docstring = inspect.getdoc(func)

    # Assuming the first line of the docstring is the function description
    function_description = docstring.split("\n")[0]

    # Extracting parameter information
    parameters = {}
    for name, param in signature.parameters.items():
        # Assume the description is in the format: `name: description`
        param_description = [
            line.split(": ")[1]
            for line in docstring.split("\n")
            if line.startswith(name + ":")
        ]
        param_description = param_description[0] if param_description else ""

        # Building the parameter info
        param_type = type_to_str(param.annotation)
        param_info = {"description": param_description}
        if isinstance(param_type, dict):
            # If the type is a dictionary (e.g., for enum), merge it with param_info
            param_info.update(param_type)
        else:
            param_info["type"] = param_type

        parameters[name] = param_info

    # Required parameters are those without default values
    required_params = [
        name
        for name, param in signature.parameters.items()
        if param.default == param.empty
    ]

    # Constructing the final description
    result = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": function_description,
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required_params,
            },
        },
    }

    return result


# Example function to describe
def get_current_weather(
    location: str, format: typing.Literal["celsius", "fahrenheit"]
) -> str:
    """
    Get the current weather.

    :param location: The city and state, e.g. San Francisco, CA.
    :param format: The temperature unit to use.
    """
    return "Sunny"


def python_type_to_json_type(python_type):
    """Map Python types to JSON Schema types."""
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        # Add more mappings as needed
    }
    return type_map.get(python_type, "any")


def type_to_str(type_hint):
    """Convert type hints to JSON-friendly string representations."""
    if type_hint == inspect.Parameter.empty:
        return "any"
    if getattr(type_hint, "__origin__", None) is typing.Literal:
        # Handling typing.Literal to convert it to JSON enum format
        return {"enum": list(type_hint.__args__)}
    if hasattr(type_hint, "__origin__"):  # For handling generic types like List[str]
        origin = type_hint.__origin__
        if origin is list:
            # Assuming only simple types like List[str], not nested like List[List[str]]
            args = type_hint.__args__[0]
            return f"array of {python_type_to_json_type(args)}"
        # Handle other generic types (like Dict, Tuple) here as needed
    return python_type_to_json_type(type_hint)


# Using the function
# print(describe_function(polite_function))
tools = [
    describe_function(polite_function),
    describe_function(impolite_function),
    describe_function(get_current_weather),
]


In [ ]:
describe_function(get_current_weather)


In [ ]:
asdfasdfasdfasdf


In [ ]:
from llamabot.components.tools import Tools


tools = Tools(polite_function, impolite_function, get_current_weather)


In [ ]:
client = OpenAI()


In [ ]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a polite robot."},
        {"role": "system", "content": "Always pick the polite function."},
        {
            "role": "user",
            "content": "What is the weather in my area, Cambridge, MA?",
        },
    ],
    tools=tools.schemas(),
)


In [ ]:
response.choices[0]


In [ ]:
import tiktoken
from llamabot.components.messages import BaseMesasge


class History:
    def __init__(self):
        self.messages: list[BaseMesasge] = []

    def append(self, message):
        self.messages.append(message)

    def retrieve(self, token_budget: int, model_name: str):
        """Retrieve messages from the history up to the token budget."""
        tokenizer = tiktoken.encoding_for_model(model_name)
        tokens = 0
        messages = []
        for message in self.messages:
            tokens += tokenizer(message).input_ids.shape[1]
            if tokens > token_budget:
                # trim the message until it fits.
                message = message[: -(tokens - token_budget)]
            messages.append(message)
        return messages

    def __getitem__(self, index):
        return self.messages[index]


class RAGHistory:
    def __init__(self):
        self.messages: list[BaseMesasge] = []

    def append(self, message):
        # Embed the message content inside an in-memory vector store for RAG.
        self.messages.append(message)

    # def retrieve(self, )


In [ ]:
tools(response.choices[0].message.tool_calls)


In [ ]:
message = SystemMessage(content="You are a polite robot.")
message.model_dump()


In [ ]:
from llamabot import SimpleBot


In [ ]:
bot = SimpleBot("You are a polite robot.")
bot("What is the weather in my area, Cambridge, MA?")


In [ ]:
AIMessage(content=response.choices[0].message.content)


In [ ]:
response.choices[0]


In [ ]:
tools(response.choices[0].message.tool_calls)


In [ ]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

# if tool_calls:
#     # Step 3: call the function
#     # Note: the JSON response may not always be valid; be sure to handle errors
#     messages.append(response_message)  # extend conversation with assistant's reply


In [ ]:
tool_calls


In [ ]:
func = available_functions[tool_calls[0].function.name]
import json

func_kwargs = json.loads(tool_calls[0].function.arguments)
func(**func_kwargs)
